### This is the first draft, just going to tokenize to notes, and then try clustering

In [1]:
# import cell
import pandas as pd
from music21 import converter, key, pitch, interval, note
import re
from abctool import abctool

In [2]:
df = pd.read_csv('/home/devcontainers/git/Tune_Similarity/raw_data/tunes.csv')
df.head()

,tune_id,setting_id,name,type,meter,mode,abc,date,username
0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler
1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles
2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie
3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh
4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK


In [8]:
df["full_abc"] = df.apply(
    lambda row: f"""X:{row.setting_id}
T:{row["name"]}
M:{row.meter}
K:{row["mode"]}
{row.abc}
""",
    axis=1
)


In [6]:
def sanitize_abc(abc_str: str) -> str:
    # Remove ornaments like {a}, {gf}, etc.
    abc_str = re.sub(r"\{[^}]*\}", "", abc_str)

    # Remove slashes that could represent strange tuplets or slashed notes
    abc_str = re.sub(r"(?<![A-Ga-g])/+", "/", abc_str)  # keep note slashes like A/B

    # Optionally: remove standalone non-note letters like "J", "X" that break parsing
    abc_str = re.sub(r"\b[J-Zj-z]\b", "", abc_str)  # single-letter tokens that aren’t notes

    return abc_str

def parse_and_transpose_score(row, target_tonic="C"):

    try:
        abc = row['full_abc']
        mode_str = row['mode']
        abc = sanitize_abc(abc)
        print(abc)
        # Parse mode string like "Gmajor" or "Ddor"
        match = re.match(r"([A-Ga-g][#b]?)(.*)", mode_str)
        if not match:
            raise ValueError(f"Invalid mode string: {mode_str}")
        tonic_str, mode = match.groups()
        tonic = pitch.Pitch(tonic_str.upper())
        mode = mode.lower()
        
        # Transpose to target tonic (e.g. "C" for major, "D" for dorian)
        score = converter.parse(abc, format='abc')
        interval_to_target = interval.Interval(tonic, pitch.Pitch(target_tonic))
        transposed = score.transpose(interval_to_target)

        return transposed

    except Exception as e:
        print(f"Failed to parse/transpose row {row.name}: {e}")
        return None


In [21]:
from abctool import abctune

abc_str = df.loc[0, "full_abc"]
tune = abctune(abc_str)

print(abc_str)

X:35234
T:$150 Boot, The
M:2/4
K:Gmajor
|:d>g fe|dB AG|E/F/G E/F/G|BA GF|
d>g fe|dB AG|E/F/G E/F/G|BA G2:|
|:e>f ga|ba gf|e/f/g e/f/g|d/c/B/A/ BA|
e>f ga|ba gf|e/f/g e/f/g|ag g2:|



In [22]:
tune.transpose(1)

print(tune.getAbc())

X:35234
T:$150 Boot, The
M:2/4
K:Abmajor
|:e>a gf|ec BA|F/G/A F/G/A|cB AG|
e>a gf|ec BA|F/G/A F/G/A|cB A2:|
|:f>g ab|c'b ag|f/g/a f/g/a|e/d/c/B/ cB|
f>g ab|c'b ag|f/g/a f/g/a|ba a2:|


In [17]:
mode = df.loc[0, "mode"]
match = re.match(r"([A-Ga-g][#b]?)(.*)", mode)
if not match:
    raise ValueError(f"Invalid mode string: {mode}")
tonic_str, mode = match.groups()
print(tonic_str)

G


In [23]:
df["mode"].value_counts()

mode
Gmajor         13307
Dmajor         13036
Amajor          3585
Adorian         3014
Eminor          2832
Edorian         2379
Bminor          1977
Amixolydian     1677
Aminor          1615
Dmixolydian     1477
Cmajor          1212
Fmajor           856
Gminor           728
Dminor           721
Ddorian          587
Gdorian          424
Emajor           313
Gmixolydian      201
Bdorian          156
Cdorian          143
Fdorian          116
Emixolydian       86
Bmixolydian       33
Name: count, dtype: int64

In [ ]:
def get_transpose_distance(m):
    # m - mode of the tune
    # returns distance to transpose so that tonic is C
    # proper way to do this would be to do regex to match any # or b, but we don't have those keys in the dataset
    tonic = m[0]
    # again, being lazy
    distance = {
        "A" : 3,
        "B" : 1,
        "C" : 0,
        "D" : -2,
        "E" : -4,
        "F" : -5,
        "G" : -7
    }
    return distance[tonic]

In [ ]:


def transpose_row(row):
    m = row["mode"]
    abc = row["full_abc"]
    tune = abctune(abc)
    d = get_transpose_distance(m)
    tune.transpose(d)
    return tune.getAbc()


df["transposed_abc"] = df.apply(transpose_row, axis = 1)

AttributeError: module 'string' has no attribute 'lower'

In [38]:
df.head()

,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc
0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:..."
1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...
2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...
3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...
4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...


In [39]:
i = 2
abc_str = df.loc[i, "full_abc"]
m = df.loc[i, "mode"]
tune = abctune(abc_str)
print(tune.getAbc())
tune.transpose(get_transpose_distance(m))
print(tune.getAbc())

X:48772
T:'G Iomain Nan Gamhna
M:9/8
K:Amixolydian
|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG A2A:|
BGB fge fed|BGB fge f3|BGB fge fed|e2A ABd e2d|
BGB fge fed|BGB fga f2e|def gfe fed|e2A ABd efg||

X:48772
T:'G Iomain Nan Gamhna
M:9/8
K:Cmixolydian
|:fdd ddc dcB|fdd def gab|fdd ddc dcB|g2c fdB c2c:|
dBd abg agf|dBd abg a3|dBd abg agf|g2c cdf g2f|
dBd abg agf|dBd abc' a2g|fga bag agf|g2c cdf gab||


In [ ]:
# normalize

In [ ]:
# normalize

In [ ]:
# cluster